In [1]:
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from surprise import Dataset, NormalPredictor, Reader, SVD
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import spacy
import textacy
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pkl
import stellargraph as sg
import tensorflow as tf
from stellargraph.data import BiasedRandomWalk
from gensim.models import Word2Vec
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from stellargraph import datasets
from IPython.display import display, HTML
from sklearn.cluster import KMeans
from scipy import stats
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from scipy import stats
from sklearn.metrics import roc_curve, auc

In [85]:
df0 = pd.read_csv("Data/cancer_dataset.csv", encoding='latin1', index_col=0)
df = df0["a"]

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df0['0'])
set(labels)

{0, 1, 2}

In [86]:
with open("Data/topics_clf.pickle", "rb") as f:
    topics = pkl.load(f)
topics = np.array(topics)
max(topics)

278

In [87]:
ratings_dict = {
    "itemID": np.array(df),
    "userID": topics,
    "rating": labels,
}
df_recommender = pd.DataFrame(ratings_dict)

reader = Reader(rating_scale=(0, 2))
data = Dataset.load_from_df(df_recommender[["userID", "itemID", "rating"]], reader)

algo = SVD()
cross_validate(algo, data, cv=5, verbose=True);

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.1255  0.1212  0.1212  0.1274  0.1218  0.1234  0.0025  
MAE (testset)     0.0841  0.0806  0.0806  0.0861  0.0809  0.0825  0.0023  
Fit time          0.08    0.09    0.09    0.09    0.10    0.09    0.01    
Test time         0.02    0.01    0.00    0.02    0.01    0.01    0.01    


In [88]:
algo.fit(data.build_full_trainset())

In [89]:
uid = str(np.array(data.raw_ratings)[:, 0][7569])
iid = str(np.array(data.raw_ratings)[:, 1][7569])#str(26)

pred = algo.predict(uid, iid, r_ui=None)
print(pred.est)

0.8239113910562115


In [92]:
y_pred = []
for i in range(len(df_recommender)):
    uid = str(np.array(data.raw_ratings)[:, 0][i])
    iid = str(np.array(data.raw_ratings)[:, 1][i])
    y = algo.predict(uid, iid, r_ui=np.array(data.raw_ratings)[:, 2][i])
    y_pred.append(y.est)

In [93]:
def normalize(value, min_value, max_value):
    return (value - min_value) / (max_value - min_value)

def map_to_discrete(normalized_value):
    if normalized_value < 1/3:
        return 0
    elif normalized_value < 2/3:
        return 1
    else:
        return 2

In [94]:
min_val, max_val = min(y_pred), max(y_pred)

In [95]:
y_pred0 = []
for i in range(len(df_recommender)):
    uid = str(np.array(data.raw_ratings)[:, 0][i])
    iid = str(np.array(data.raw_ratings)[:, 1][i])
    y = algo.predict(uid, iid, r_ui=np.array(data.raw_ratings)[:, 2][i])
    
    normalized_value = normalize(y.est, min_val, max_val)
    discrete_value = map_to_discrete(normalized_value)
    
    y_pred0.append(discrete_value)

In [96]:
accuracy_score(y_pred0, labels)

0.8891677675033025

In [97]:
#

In [98]:
with open("Data/graphs.pkl", "rb") as f:
    Gs = pkl.load(f)
GG = nx.disjoint_union_all(Gs)

with open("Data/combined_graphed.pkl", "rb") as f:
    node_vectors_list_cmb = pkl.load(f)

with open("Data/individual_graphed.pkl", "rb") as f:
    node_vectors_list_ind = pkl.load(f)
    
word_dict = {}
i = 0
for G in Gs:
    for node in list(G.nodes):
        word_dict[i] = node
        i += 1
        
len_each_graph = [len(i) for i in node_vectors_list_ind]
print(len_each_graph[:20])

[88, 76, 91, 90, 117, 125, 94, 106, 188, 203, 182, 170, 167, 183, 203, 45, 144, 185, 182, 161]


In [99]:
vec_list = np.array(node_vectors_list_cmb[0])
len(vec_list)

1009270

In [100]:
#

In [105]:
topics_extended = []
for i, l in enumerate(len_each_graph):
    topics_extended.extend([y_pred0[i]]*l)

In [106]:
labels_extended = []
for i, l in enumerate(len_each_graph):
    labels_extended.extend([labels[i]]*l)

In [107]:
differences_index = []
for i, l in enumerate(labels_extended):
    if l != topics_extended[i]:
        differences_index.append(i)
len(differences_index)

139621

In [108]:
inconsistent_words = []
for i in differences_index:
    inconsistent_words.append(word_dict[i])

In [109]:
#

In [132]:
X_train, X_test, y_train, y_test = train_test_split(
    vec_list, labels_extended, test_size=0.1
)
#model = LinearSVC()
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [133]:
y_pred = model.predict(X_test)
accuray = accuracy_score(y_pred, y_test)
f1 = f1_score(y_pred, y_test, average="weighted")

print("Accuracy:", accuray)
print("F1 Score:", f1)

Accuracy: 0.4819324858561138
F1 Score: 0.5302977769409778


In [118]:
#

In [119]:
vec_list_reduced = np.delete(vec_list, differences_index, axis=0)
bert_reduced = np.delete(topics_extended, differences_index)
labels_extended_reduced = np.delete(labels_extended, differences_index)

In [128]:
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    vec_list_reduced, labels_extended_reduced, test_size=0.3
)
#model_r = LinearSVC()
model_r = GaussianNB()
model_r.fit(X_train_r, y_train_r)

GaussianNB()

In [129]:
y_pred_r = model_r.predict(X_test_r)
accuray = accuracy_score(y_pred_r, y_test_r)
f1 = f1_score(y_pred_r, y_test_r, average="weighted")

print("Accuracy:", accuray)
print("F1 Score:", f1)

Accuracy: 0.44875141340385977
F1 Score: 0.4942689790826937


In [23]:
#